In [1]:
import os
import json
import numpy as np
import albumentations as A
import cv2
import pandas as pd
import sys
sys.path.append(os.path.abspath("../code_base"))
from _info_ import ear_types, degrees, acupoints_name, cm, rotation_angles
from _common_ import angles, split_xy_xyv
# from matplotlib.font_manager import FontProperties
# from ..code_base._info_ import ear_types, degrees, acupoints_name, cm, rotation_angles
# from ..code_base._common_ import angles, split_xy_xyvm


In [8]:
class rotation_img():
    def __init__(self):
        self.kpt_folder = "../keypoint"
        self.rotation_img = "../rotation_img"
        self.rotation_csv = "../rotation_csv"
    def readKeypoints(self, frameKpts):
        final = []
        for i in range(0, len(frameKpts)):
            x = int(frameKpts[str(i)]["keypoint_location"][0])
            y = int(frameKpts[str(i)]["keypoint_location"][1])
            final.append((x,y))
        return final
    def readVis(self, frameVis):
        final = []
        for i in range(0, len(frameVis)):
            final.append(int(frameVis[str(i)]["visible"]))
        return final
        
    def rotate_image(self, image_path, angle, keypoints, visible):
        # 读取图像
        image = cv2.imread(image_path)
        h, w = image.shape[:2]


        # 旋转中心
        center = (w / 2, h / 2)

        # 计算旋转所需的最小画布尺寸
        radians = np.radians(angle)

        sin, cos = np.sin(radians), np.cos(radians)
        bound_w = int((h * abs(sin)) + (w * abs(cos)))
        bound_h = int((h * abs(cos)) + (w * abs(sin)))

        # 旋转变换
        if sin == 0.0:
            image = cv2.imread(image_path)
            return image, keypoints

        else:
            transform = A.Compose([
                # A.PadIfNeeded(min_height=bound_h, min_width=bound_w, border_mode=cv2.BORDER_REPLICATE, p=1.0),
                A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_REPLICATE, p=1.0)
                ],
                keypoint_params=A.KeypointParams(format='xy', remove_invisible=False, angle_in_degrees=True),
             p=1)

            # 应用变换
            transformed = transform(image=image, keypoints=keypoints)
            transformed_image = transformed["image"]
            transformed_keypoints = transformed['keypoints']
            return transformed_image, transformed_keypoints
        
    def generate(self):
        for angle in rotation_angles:
            for ear_type in ear_types:
                names = os.listdir(os.path.join(self.kpt_folder, ear_type, "4_return_to_ori_size"))
                for name in names:
                    for deg in degrees:
                        kpt_file = os.path.join(self.kpt_folder, ear_type, "2_json", name, deg, "keypoint_location.json")
                        vis_file = os.path.join(self.kpt_folder, ear_type, "2_json", name, deg, "visible.json")
                        with open(kpt_file, 'r') as f1:
                            kpts = json.load(f1)
                        with open(vis_file, 'r') as f2:
                            vis = json.load(f2)

                        indexes = angles(kpts, 4)

                        gt_df = pd.DataFrame(columns=list(range(len(kpts["frame_0"]))), index = indexes)

                        for index in indexes:
                            keypoints = self.readKeypoints(kpts["frame_"+str(index)])
                            visible = self.readVis(vis["frame_"+str(index)])
                            img_path = os.path.join(self.kpt_folder, ear_type, "4_return_to_ori_size", name, deg, str(index)+".png")

                            transformed_image, transformed_keypoints = self.rotate_image(img_path, angle, keypoints, visible)
                            for j in range(0, len(transformed_keypoints)):
                                if transformed_keypoints[j][0] > 0 and transformed_keypoints[j][1] > 0:
                                    gt_df.loc[index,j] = str(transformed_keypoints[j][0])+","+str(transformed_keypoints[j][1]) +"," + str(visible[j])
                                else:
                                    gt_df.loc[index,j] = str(transformed_keypoints[j][0])+","+str(transformed_keypoints[j][1]) +"," + "0"

                            transformed_path = os.path.join(self.rotation_img, ear_type, name, deg, str(angle),"img")
                            if not os.path.exists(transformed_path):
                                os.makedirs(transformed_path)

                            save_path = os.path.join(transformed_path, "{index}.png".format(index = index))
                            cv2.imwrite(save_path, transformed_image)
                        if not os.path.exists(os.path.join(self.rotation_csv, ear_type, name, deg, str(angle))):
                            os.makedirs(os.path.join(self.rotation_csv, ear_type, name, deg, str(angle)))
                        gt_df.to_csv(os.path.join(self.rotation_csv, ear_type, name, deg, str(angle), "gt.csv"))


        

In [9]:
B = rotation_img()
B.generate()

In [3]:
kpt="../keypoint"
data_aug = "../data_aug"

In [18]:
# def rotate_image_without_cropping(image_path, angle, keypoints, visible):
#     # 读取图像
#     image = cv2.imread(image_path)
#     h, w = image.shape[:2]


#     # 旋转中心
#     center = (w / 2, h / 2)

#     # 计算旋转所需的最小画布尺寸
#     radians = np.radians(angle)
    
#     sin, cos = np.sin(radians), np.cos(radians)
#     bound_w = int((h * abs(sin)) + (w * abs(cos)))
#     bound_h = int((h * abs(cos)) + (w * abs(sin)))

#     # 旋转变换
#     if sin == 1.0:
#         transform = A.Compose([
#         # A.PadIfNeeded(min_height=bound_h, min_width=bound_w, border_mode=cv2.BORDER_REPLICATE, p=1.0),
#         A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_REPLICATE, p=1.0)
#         ],
#         keypoint_params=A.KeypointParams(format='xy', remove_invisible=False, angle_in_degrees=True),
#      p=1)

#     # 应用变换
#         transformed = transform(image=image, keypoints=keypoints)
#         transformed_image = transformed["image"][:][210:750]
#         transformed_keypoints = []
#         for kpt in transformed['keypoints']:
#             transformed_keypoints.append((kpt[0],kpt[1]-210))
    
#         return transformed_image, transformed_keypoints

#     else:
#         transform = A.Compose([
#             # A.PadIfNeeded(min_height=bound_h, min_width=bound_w, border_mode=cv2.BORDER_REPLICATE, p=1.0),
#             A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_REPLICATE, p=1.0)
#             ],
#             keypoint_params=A.KeypointParams(format='xy', remove_invisible=False, angle_in_degrees=True),
#          p=1)
    
#         # 应用变换
#         transformed = transform(image=image, keypoints=keypoints)
#         transformed_image = transformed["image"]
#         transformed_keypoints = transformed['keypoints']
#         return transformed_image, transformed_keypoints

In [15]:
# import cv2
# import numpy as np



# # 讀取影像
# image = cv2.imread(r'D:\keypoint\free\4_return_to_ori_size\cather\15cm_50mm_10deg\0.png')
# # 旋轉影像
# rotated_image = rotate_image_without_cropping(r'D:\keypoint\free\4_return_to_ori_size\cather\15cm_50mm_10deg\0.png', 30)
# # 顯示旋轉後的影像
# cv2.imshow('Rotated Image without Cropping', rotated_image)
# cv2.imwrite("try.png", rotated_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()